# 1. Inroduction

## 1.1. Problem statement

# 2. Importing Libraries

These are collection of functions and methods that allows for performance of many actions without writing the code.

In [4]:
# reading in data
import re
import string
import numpy as np 
import random
import pandas as pd 

#Vissualizations
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go # pip install plotly==4.8.1
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator #pip install wordcloud

#Preprocessing
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
import os
import nltk
nltk.download('averaged_perceptron_tagger')
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

#modeling
from sklearn.feature_extraction.text import CountVectorizer

#csv paths
import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\elton\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Below is a helper Function which generates random colors which will be used to give different colors to the plots.

In [5]:
def random_colours(number_of_colors):
    '''
    Simple function for random colours generation.
    Input:
        number_of_colors - integer value indicating 
        the number of colours which are going to be 
        generated.
    Output:
        Color in the following format: ['#E86DA4'] .
    '''
    colors = []
    for i in range(number_of_colors):
        colors.append("#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]))
    return colors

# 3. Getting Data

The data is given in the format csv files, pandas are used to import and convert the csv files to panda data frame.

In [6]:
# Load dataset
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
ss_df = pd.read_csv('sample_submission.csv')

# 4. Data Desciption

Below is just the over view of the data frames and thier contents.

In [ ]:
print('This is the Train data frame')
print(train_df.head(4))
print('')
print('This is the Test data frame')
print(test_df.head(4))

Both data frames have message and tweetid columns, the difference is that the train data frame has sentiment column and test data frame does not.

In [ ]:
ss_df.head(4)

Above is the Sample submission data frame, which is just a guide on how the submission should be made. It has only two columns, the tweetid and the sentiment.

In [ ]:
print('___basic info of the train data___')
print(train_df.info())
print('Dataset size:', train_df.shape)
print('Columns are:',train_df.columns)


Looking at the train dataframe basic information, it shows that there are 15819 entries, 0 to 15818.The sentiment and the tweetid are intigers and the sentiment is an object. so the shape is 15819 entries by 3 columns

# 5 Exploratory Data Analysis on Raw data

## 5.1. Taking a closer look at the distribution of tweeter messages in the train set.

In [ ]:
temp = train_df.groupby('sentiment').count()['message'].reset_index().sort_values(by='message',ascending=False)
temp['percentage'] = round((temp['message']/temp['message'].sum())*100,0)
temp.style.background_gradient(cmap='Purples')

In [ ]:
fig = px.funnel(temp, x='sentiment', y='message',color='sentiment')
fig.show()

The table and the funnel display the count and percentage of message in the train data frame. Sentiment 1 has the highest count of message of 8530 which is 54% of the whole message and negative sentiment (-1) has the lowest count of 1296 which is 8% of the whole messages. 

## 5.2. Wordcloud Visualization

This is a popular visualization of words.

In [ ]:
#pd.set_option('display.max_colwidth', 60)
# visualising the messages 
df_senti1 = train_df[train_df['sentiment']==1]
df_senti0 = train_df[train_df['sentiment']==0]
df_senti_neg1 = train_df[train_df['sentiment']==-1]
df_senti2 = train_df[train_df['sentiment']==2]
tweet_All = " ".join(review for review in train_df.message)
tweet_senti1 = " ".join(review for review in df_senti1.message)
tweet_senti0 = " ".join(review for review in df_senti0.message)
tweet_senti_neg1 = " ".join(review for review in df_senti_neg1.message)
tweet_senti2 = " ".join(review for review in df_senti2.message)

fig, ax = plt.subplots(5, 1, figsize  = (65,65))
# Create and generate a word cloud image:
wordcloud_ALL = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_All)
wordcloud_1 = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_senti1)
wordcloud_0 = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_senti0)
wordcloud_neg1 = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_senti_neg1)
wordcloud_2 = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_senti2)

# Display the generated image:
ax[0].imshow(wordcloud_ALL, interpolation='bilinear')
ax[0].set_title('All Tweets', fontsize=50)
ax[0].axis('off')
ax[1].imshow(wordcloud_1, interpolation='bilinear')
ax[1].set_title('Tweets under Pro Class 1',fontsize=50)
ax[1].axis('off')
ax[2].imshow(wordcloud_0, interpolation='bilinear')
ax[2].set_title('Tweets under Neutral Class 0',fontsize=50)
ax[2].axis('off')
ax[3].imshow(wordcloud_neg1, interpolation='bilinear')
ax[3].set_title('Tweets under Anti Class -1',fontsize=50)
ax[3].axis('off')
ax[4].imshow(wordcloud_2, interpolation='bilinear')
ax[4].set_title('Tweets under News Class 2',fontsize=50)
ax[4].axis('off')

The Wordcloud Visualization display top 100 words from the whole message column and semtimentwise.The size of the words is related to how frequent the word occured in the messages,the bigger the word the more frequent it appeared in the messages.

* All tweet message 
some of the most occuring words in all tweets are 'climate change, is, https, CO, RT and global warming'

* Tweets messages under Pro sentiment(1)
most common words here 'global warming, https, RT,CO Believe, doesnt believe'

* tweets under sentiment 0 the Neutral
most common words are global warming, CO, Https,Rt
* tweets under sentiment -1 the Anti
common words oare 'Co,RT,Https, Trump, man  made, global warming'
* tweets under sentiment 2 the News
common words are 'Co,RT,Https, Trump, global warming, via

This is interesting because some of the words such as belive ,Trump are only common in certain sentiments. the RT refers to retweets and the https is the hyper link on the tweet.

It can be concluded that there are lot of retweets in the data frame. Retweets are just repetions of an original post on twitter shared by different users. This means that a person sharing the retweet is just supporting the origional post. the name Trump which is the name of US president seems to appear more often awell. Words such as climate, global warming related to the problem statement of this project unlike other words such as 'via, is, https' which are frequently occuring from the tweets.




### Checking top common words by count

diving deeper to check count of some common words from the tweet messages

In [ ]:
# Checking top 20 common words from all tweets
train_df['temp_list'] = train_df['message'].apply(lambda x:str(x).split())
top = Counter([item for sublist in train_df['temp_list'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Blues')

In [ ]:
fig = px.bar(temp, x="count", y="Common_words", title='Commmon Words in Raw message', orientation='h', 
             width=700, height=700,color='Common_words')
fig.show()

Climate RT and Change are still top most occuring words in the tweets.Whats new here is that common words such as 'The,to, of ,is ,a'are now showeing to be appearing more often aswell. 

In [ ]:
#defining the sentiments so that each sentiment can be explored
Pro = train_df[train_df['sentiment']==1]
Anti = train_df[train_df['sentiment']== -1]
Neutral = train_df[train_df['sentiment']==0]
News = train_df[train_df['sentiment']== 2]

In [ ]:
#MosT common positive words(Pro)
top = Counter([item for sublist in Pro['temp_list'] for item in sublist])
temp_positive = pd.DataFrame(top.most_common(20))
temp_positive.columns = ['Common_words','count']
temp_positive.style.background_gradient(cmap='Oranges')

# 6 Data cleaning and Preprocessing

In [ ]:
#MosT common negative words(Anti)
top = Counter([item for sublist in Anti['temp_list'] for item in sublist])
temp_negative = pd.DataFrame(top.most_common(20))
temp_negative = temp_negative.iloc[1:,:]
temp_negative.columns = ['Common_words','count']
temp_negative.style.background_gradient(cmap='Reds')

In [ ]:
#MosT common words in Neutral sentiment
top = Counter([item for sublist in Neutral['temp_list'] for item in sublist])
temp_neutral = pd.DataFrame(top.most_common(20))
temp_neutral = temp_neutral.loc[1:,:]
temp_neutral.columns = ['Common_words','count']
temp_neutral.style.background_gradient(cmap='Reds')

In [ ]:
#MosT common words in News sentiment
top = Counter([item for sublist in News['temp_list'] for item in sublist])
temp_neutral = pd.DataFrame(top.most_common(20))
temp_neutral = temp_neutral.loc[1:,:]
temp_neutral.columns = ['Common_words','count']
temp_neutral.style.background_gradient(cmap='Reds')

# 6. Data cleaning and Preprocessing

### 6.1. Cleaning Data

In [7]:
def cleaning(text):
    '''The Function change contraction words to thier full format(eg did'nt ti did not), lowers 
    all text to lower case format, makes 
    text lowercase,removes links(https), 
    removes punctuation(!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~)
    , removes words containing numbers and words that are attachec with #tag.'''
    
    contractions_dict = {"ain't": "am not / are not / is not / has not / have not",
                         "aren't": "are not / am not","can't": "cannot","can't've": "cannot have",
                         "'cause": "because","could've": "could have","couldn't": "could not",
                         "couldn't've": "could not have","didn't": "did not","doesn't": "does not",
                         "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                         "hasn't": "has not","haven't": "have not","he'd": "he had / he would",
                         "he'd've": "he would have","he'll": "he shall / he will",
                         "he'll've": "he shall have / he will have","he's": "he has / he is",
                         "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                         "how's": "how has / how is / how does","I'd": "I had / I would",
                         "I'd've": "I would have","I'll": "I shall / I will",
                         "I'll've": "I shall have / I will have","I'm": "I am","I've": "I have",
                         "isn't": "is not","it'd": "it had / it would","it'd've": "it would have",
                         "it'll": "it shall / it will","it'll've": "it shall have / it will have",
                         "it's": "it has / it is","let's": "let us","ma'am": "madam","mayn't": "may not",
                         "might've": "might have","mightn't": "might not","mightn't've": "might not have",
                         "must've": "must have","mustn't": "must not","mustn't've": "must not have",
                         "needn't": "need not","needn't've": "need not have","o'clock": "of the clock",
                         "oughtn't": "ought not","oughtn't've": "ought not have","shan't": "shall not",
                         "sha'n't": "shall not","shan't've": "shall not have","she'd": "she had / she would",
                         "she'd've": "she would have",
                         "she'll": "she shall / she will","she'll've": "she shall have / she will have",
                         "she's": "she has / she is","should've": "should have","shouldn't": "should not",
                         "shouldn't've": "should not have","so've": "so have","so's": "so as / so is",
                         "that'd": "that would / that had","that'd've": "that would have","that's": "that has / that is",
                         "there'd": "there had / there would","there'd've": "there would have",
                         "there's": "there has / there is","they'd": "they had / they would",
                         "they'd've": "they would have","they'll": "they shall / they will",
                         "they'll've": "they shall have / they will have","they're": "they are",
                         "they've": "they have","to've": "to have","wasn't": "was not",
                         "we'd": "we had / we would","we'd've": "we would have",
                         "we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
                         "weren't": "were not","what'll": "what shall / what will",
                         "what'll've": "what shall have / what will have",
                         "what're": "what are","what's": "what has / what is",
                         "what've": "what have","when's": "when has / when is","when've": "when have",
                         "where'd": "where did","where's": "where has / where is",
                         "where've": "where have","who'll": "who shall / who will",
                         "who'll've": "who shall have / who will have",
                         "who's": "who has / who is","who've": "who have",
                         "why's": "why has / why is","why've": "why have","will've": "will have",
                         "won't": "will not","won't've": "will not have","would've": "would have",
                         "wouldn't": "would not","wouldn't've": "would not have",
                         "y'all": "you all","y'all'd": "you all would",
                         "y'all'd've": "you all would have","y'all're": "you all are",
                         "y'all've": "you all have","you'd": "you had / you would",
                         "you'd've": "you would have","you'll": "you shall / you will",
                         "you'll've": "you shall have / you will have","you're": "you are",
                         "you've": "you have"}

    for word in text.split():
        if word.lower() in contractions_dict:
            text = text.replace(word,contractions_dict[word.lower()])
    text = str(text).lower() #Make text lowercase
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",text).split())# remove @user, #word and link
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) ##remove punctuation
    text = re.sub('\n', '', text)
    text = ''.join([i for i in text[:] if not i.isdigit()]) #remove numbers
    return text

train_df['clean_message'] = train_df['message'].apply(lambda x: cleaning(x))
test_df['clean_message'] = test_df['message'].apply(lambda x: cleaning(x))

In [8]:
train_df.head()

,sentiment,message,tweetid,clean_message
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221,polyscimajor epa chief does not think carbon d...
1,1,It's not like we lack evidence of anthropogeni...,126103,it has it is not like we lack evidence of anth...
2,2,RT @RawStory: Researchers say we have three ye...,698562,rt researchers say we have three years to act ...
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736,wired was a pivotal year in the war on climat...
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954,rt it has it is and a racist sexist climate c...


## 6.2. Preprocessing

### 6.2.1. Lemmatisation


The purpose of Lemmatization is to remove inflectional endings only and to return the base or dictionary form of a word, which is known as the lemma.

In [9]:
nlp = spacy.load("en_core_web_lg")
def lemmatizer(text):        
    sent = []
    doc = nlp(text)
    for word in doc:
        sent.append(word.lemma_)
    return " ".join(sent)

In [10]:
train_df["lemmaa"] =train_df.apply(lambda x: lemmatizer(x['clean_message']), axis=1)
test_df["lemmaa"] =test_df.apply(lambda x: lemmatizer(x['clean_message']), axis=1)

In [12]:
train_df.head() 

,sentiment,message,tweetid,clean_message,lemmaa
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221,polyscimajor epa chief does not think carbon d...,polyscimajor epa chief do not think carbon dio...
1,1,It's not like we lack evidence of anthropogeni...,126103,it has it is not like we lack evidence of anth...,-PRON- have -PRON- be not like -PRON- lack evi...
2,2,RT @RawStory: Researchers say we have three ye...,698562,rt researchers say we have three years to act ...,rt researcher say -PRON- have three year to ac...
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736,wired was a pivotal year in the war on climat...,wired be a pivotal year in the war on climat...
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954,rt it has it is and a racist sexist climate c...,rt -PRON- have -PRON- be and a racist sexist...


### 6.2.2. Removing the PRON and rt form column lemmaa

In [13]:
def remove(text):
    '''removing the words -PRON- and rt'''
    text = re.sub(r'-PRON-','',text)
    text = re.sub(r'rt' , '' , text) 
    return text

In [14]:
train_df['clean_lemma'] = train_df['lemmaa'].apply(lambda x: remove(x))
test_df['clean_lemma'] = test_df['lemmaa'].apply(lambda x: remove(x))
train_df.head()

,sentiment,message,tweetid,clean_message,lemmaa,clean_lemma
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221,polyscimajor epa chief does not think carbon d...,polyscimajor epa chief do not think carbon dio...,polyscimajor epa chief do not think carbon dio...
1,1,It's not like we lack evidence of anthropogeni...,126103,it has it is not like we lack evidence of anth...,-PRON- have -PRON- be not like -PRON- lack evi...,have be not like lack evidence of anthropog...
2,2,RT @RawStory: Researchers say we have three ye...,698562,rt researchers say we have three years to act ...,rt researcher say -PRON- have three year to ac...,researcher say have three year to act on cli...
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736,wired was a pivotal year in the war on climat...,wired be a pivotal year in the war on climat...,wired be a pivotal year in the war on climat...
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954,rt it has it is and a racist sexist climate c...,rt -PRON- have -PRON- be and a racist sexist...,have be and a racist sexist climate chang...


### 6.2.3. Tokenizing

In [15]:
from nltk.tokenize import word_tokenize
def tokenization(text):
    '''split the message into taken of words'''
    text = word_tokenize(text)
    return text

In [16]:
train_df['tokenized_message'] = train_df['clean_lemma'].apply(lambda x: tokenization(x.lower()))
test_df['tokenized_message'] = test_df['clean_lemma'].apply(lambda x: tokenization(x.lower()))
train_df.head()

,sentiment,message,tweetid,clean_message,lemmaa,clean_lemma,tokenized_message
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221,polyscimajor epa chief does not think carbon d...,polyscimajor epa chief do not think carbon dio...,polyscimajor epa chief do not think carbon dio...,"[polyscimajor, epa, chief, do, not, think, car..."
1,1,It's not like we lack evidence of anthropogeni...,126103,it has it is not like we lack evidence of anth...,-PRON- have -PRON- be not like -PRON- lack evi...,have be not like lack evidence of anthropog...,"[have, be, not, like, lack, evidence, of, anth..."
2,2,RT @RawStory: Researchers say we have three ye...,698562,rt researchers say we have three years to act ...,rt researcher say -PRON- have three year to ac...,researcher say have three year to act on cli...,"[researcher, say, have, three, year, to, act, ..."
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736,wired was a pivotal year in the war on climat...,wired be a pivotal year in the war on climat...,wired be a pivotal year in the war on climat...,"[wired, be, a, pivotal, year, in, the, war, on..."
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954,rt it has it is and a racist sexist climate c...,rt -PRON- have -PRON- be and a racist sexist...,have be and a racist sexist climate chang...,"[have, be, and, a, racist, sexist, climate, ch..."


### 6.2.4. Stop Words

In [17]:
stop = ['a', 'about', 'above','after', 'again', 'against', 'ain', 'all', 
        'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 
        'be', 'because', 'been', 'before', 'being','below', 'between', 
        'both', 'but', 'by', 'can', 'couldn',"couldn't", 'd', 'did', 'didn', 
        "didn't", 'do', 'does', 'doesn',"doesn't", 'doing', 'don', "don't",
        'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', 
        "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 
        'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 
        'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 
        'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 
        'my', 'myself', 'needn', "needn't", 'no', 'nor','now', 'o', 'of', 'off', 
        'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 
        're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', 
        "shouldn't", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 
        'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 
        'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'wasn', "wasn't", 
        'we', 'were', 'weren', "weren't", 'what', 'when', 'where', 'which', 'while', 'who', 
        'whom', 'why', 'will', 'with', 'won', "won't", 'wouldn', "wouldn't", 'y', 'you', 
        "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']

The word not is kept out of the stop words, this is because the word can help the mechine lean on the negative sentiment.

In [18]:
train_df['clean_stp_words'] = train_df['tokenized_message'].apply(lambda x: [item for item in x if item not in stop])
test_df['clean_stp_words'] = test_df['tokenized_message'].apply(lambda x: [item for item in x if item not in stop])
train_df.head()

,sentiment,message,tweetid,clean_message,lemmaa,clean_lemma,tokenized_message,clean_stp_words
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221,polyscimajor epa chief does not think carbon d...,polyscimajor epa chief do not think carbon dio...,polyscimajor epa chief do not think carbon dio...,"[polyscimajor, epa, chief, do, not, think, car...","[polyscimajor, epa, chief, not, think, carbon,..."
1,1,It's not like we lack evidence of anthropogeni...,126103,it has it is not like we lack evidence of anth...,-PRON- have -PRON- be not like -PRON- lack evi...,have be not like lack evidence of anthropog...,"[have, be, not, like, lack, evidence, of, anth...","[not, like, lack, evidence, anthropogenic, glo..."
2,2,RT @RawStory: Researchers say we have three ye...,698562,rt researchers say we have three years to act ...,rt researcher say -PRON- have three year to ac...,researcher say have three year to act on cli...,"[researcher, say, have, three, year, to, act, ...","[researcher, say, three, year, act, climate, c..."
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736,wired was a pivotal year in the war on climat...,wired be a pivotal year in the war on climat...,wired be a pivotal year in the war on climat...,"[wired, be, a, pivotal, year, in, the, war, on...","[wired, pivotal, year, war, climate, change]"
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954,rt it has it is and a racist sexist climate c...,rt -PRON- have -PRON- be and a racist sexist...,have be and a racist sexist climate chang...,"[have, be, and, a, racist, sexist, climate, ch...","[racist, sexist, climate, change, deny, bigot,..."


### 6.2.5. Detokinization

In [20]:
from nltk.tokenize.treebank import TreebankWordDetokenizer as Detok

def detokenization(text):
    '''counter slpit the message into taken of words'''
    detokenizer = Detok()
    text = detokenizer.detokenize(text)
    return text
train_df['detokenized'] = train_df['clean_stp_words'].apply(lambda x: detokenization(x))
test_df['detokenized'] = test_df['clean_stp_words'].apply(lambda x: detokenization(x))

# 7. Creating features

## 7.1. Getting the number of clean words and comparing them with that  raw from the message and clean message columns.

In [ ]:
#train data frame
train_df['Num_words_raw'] = train_df['message'].apply(lambda x:len(str(x).split())) 
train_df['Num_words_clean'] = train_df['clean_message'].apply(lambda x:len(str(x).split()))
#Difference in Number of words text and Selected Text
train_df['difference_in_wordsNo'] = abs(train_df['Num_words_raw'] - train_df['Num_words_clean']) 
#test data frame
test_df['Num_words_raw'] = test_df['message'].apply(lambda x:len(str(x).split())) 
test_df['Num_words_clean'] = test_df['clean_message'].apply(lambda x:len(str(x).split())) 
#Difference in Number of words text and Selected Text
test_df['difference_in_wordsNo'] = abs(test_df['Num_words_raw'] - test_df['Num_words_clean'])

In [ ]:
train_df.head(4)  

two columns are created to show the total number of words before and after cleaning to gether with thier differences in words.

## 7.2. Jaccard score

### 7.2.1. Jaccard score compares members for two sets to see which members are shared and which are distinct. It's a measure of similarity for the two sets of data, with a range from 0% to 100%.

In [ ]:
def jaccard(str1, str2): 
    '''It takes two strings can be two columns fro a df and returns intersection of twe two  divided by thier union'''
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
# on the train_df
results_jaccard=[]

for ind,row in train_df.iterrows():
    sentence1 = row.message
    sentence2 = row.clean_message

    jaccard_score = jaccard(sentence1,sentence2)
    results_jaccard.append([sentence1,sentence2,jaccard_score])

In [ ]:
# Joining the jaccard score to the main data frame
jaccard = pd.DataFrame(results_jaccard,columns=["message","clean_message","jaccard_score"])
train_df = train_df.merge(jaccard,how='outer')

In [ ]:
train_df.head()

# 8. Exploratory Data Analysis on Clean data

## 8.1. Word Visualization of clean tweet messages

In [ ]:
#pd.set_option('display.max_colwidth', 60)
# visualising the clean messages 
df_senti1 = train_df[train_df['sentiment']==1]
df_senti0 = train_df[train_df['sentiment']==0]
df_senti_neg1 = train_df[train_df['sentiment']==-1]
df_senti2 = train_df[train_df['sentiment']==2]
tweet_All = " ".join(review for review in train_df.detokenized)
tweet_senti1 = " ".join(review for review in df_senti1.detokenized)
tweet_senti0 = " ".join(review for review in df_senti0.detokenized)
tweet_senti_neg1 = " ".join(review for review in df_senti_neg1.detokenized)
tweet_senti2 = " ".join(review for review in df_senti2.detokenized)

fig, ax = plt.subplots(5, 1, figsize  = (65,65))
# Create and generate a word cloud image:
wordcloud_ALL = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_All)
wordcloud_1 = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_senti1)
wordcloud_0 = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_senti0)
wordcloud_neg1 = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_senti_neg1)
wordcloud_2 = WordCloud(max_font_size=50, max_words=100, background_color="white").generate(tweet_senti2)

# Display the generated image:
ax[0].imshow(wordcloud_ALL, interpolation='bilinear')
ax[0].set_title('All Tweets', fontsize=50)
ax[0].axis('off')
ax[1].imshow(wordcloud_1, interpolation='bilinear')
ax[1].set_title('Tweets under Pro Class 1',fontsize=50)
ax[1].axis('off')
ax[2].imshow(wordcloud_0, interpolation='bilinear')
ax[2].set_title('Tweets under Neutral Class 0',fontsize=50)
ax[2].axis('off')
ax[3].imshow(wordcloud_neg1, interpolation='bilinear')
ax[3].set_title('Tweets under Anti Class -1',fontsize=50)
ax[3].axis('off')
ax[4].imshow(wordcloud_2, interpolation='bilinear')
ax[4].set_title('Tweets under News Class 2',fontsize=50)
ax[4].axis('off')

## 8.2. Most Common words in clean message

In [ ]:
train_df['temp_list0'] = train_df['detokenized'].apply(lambda x:str(x).split())
top = Counter([item for sublist in train_df['temp_list0'] for item in sublist])
temp0 = pd.DataFrame(top.most_common(20))
temp0.columns = ['Common_words','count']
temp0.style.background_gradient(cmap='Blues')

In [ ]:
fig = px.bar(temp0, x="count", y="Common_words", title='Commmon Words in cleaned message', orientation='h', 
             width=700, height=700,color='Common_words')
fig.show()

## 8.3. Most common words sentiment wise in clean message

In [ ]:
#defining the sentiments so that each sentiment can be explored
Pro = train_df[train_df['sentiment']==1]
Anti = train_df[train_df['sentiment']== -1]
Neutral = train_df[train_df['sentiment']==0]
News = train_df[train_df['sentiment']== 2]

In [ ]:
#MosT common positive words(Pro)
top = Counter([item for sublist in Pro['temp_list0'] for item in sublist])
temp_positive = pd.DataFrame(top.most_common(20))
temp_positive.columns = ['Common_words','count']
temp_positive.style.background_gradient(cmap='Greens')

In [ ]:
#MosT common words in Neutral sentiment
top = Counter([item for sublist in Neutral['temp_list0'] for item in sublist])
temp_neutral = pd.DataFrame(top.most_common(20))
temp_neutral = temp_neutral.loc[1:,:]
temp_neutral.columns = ['Common_words','count']
temp_neutral.style.background_gradient(cmap='Greens')

In [ ]:
#MosT common words in Neutral sentiment
top = Counter([item for sublist in Neutral['temp_list0'] for item in sublist])
temp_neutral = pd.DataFrame(top.most_common(20))
temp_neutral = temp_neutral.loc[1:,:]
temp_neutral.columns = ['Common_words','count']
temp_neutral.style.background_gradient(cmap='Reds')

In [ ]:
#MosT common words in News sentiment
top = Counter([item for sublist in News['temp_list0'] for item in sublist])
temp_neutral = pd.DataFrame(top.most_common(20))
temp_neutral = temp_neutral.loc[1:,:]
temp_neutral.columns = ['Common_words','count']
temp_neutral.style.background_gradient(cmap='Reds')

## 8.4. Number of raw and clean message columns  

## 9. Below is modeling of the data

## 9.1. SUPPORT VECTOR MACHINES/CLASSIFICATION

The objective of a Linear SVC (Support Vector Classifier) is to fit to the data provided, returning a "best fit" hyperplane that categorizes data.

In [21]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from nltk.tokenize.treebank import TreebankWordDetokenizer as Detok
import sklearn.metrics as metrics


#### Building pipelines to vectorize the data, then train and fit a model

In [22]:
# Linear SVC:
text_clf_lsvc = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

### Training on raw data

In [ ]:
# defing the x and y varible
X= train_df['message']
y=train_df['sentiment']
# slpitting the X and y into train and test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Linear SVC:
text_clf_lsvc = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])
text_clf_lsvc.fit(X_train, y_train)
# Form a prediction set
predictions = text_clf_lsvc.predict(X_test)
print(metrics.confusion_matrix(y_test,predictions))
# Print a classification report
print(metrics.classification_report(y_test,predictions))

### Training on clean message

In [23]:
# defing the x and y varible
X= train_df['detokenized']
y=train_df['sentiment']
# slpitting the X and y into train and test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Linear SVC:
text_clf_lsvc = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])
text_clf_lsvc.fit(X_train, y_train)
# Form a prediction set
predictions = text_clf_lsvc.predict(X_test)
print(metrics.confusion_matrix(y_test,predictions))
# Print a classification report
print(metrics.classification_report(y_test,predictions))

[[ 121   35  112   10]
 [  24  180  187   34]
 [  36  107 1461  151]
 [   9   23  159  515]]
              precision    recall  f1-score   support

          -1       0.64      0.44      0.52       278
           0       0.52      0.42      0.47       425
           1       0.76      0.83      0.80      1755
           2       0.73      0.73      0.73       706

    accuracy                           0.72      3164
   macro avg       0.66      0.61      0.63      3164
weighted avg       0.71      0.72      0.71      3164



In [24]:
text_clf_lsvc

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [28]:
import pickle


In [29]:
def save_model(model, file_name):
    model_save_path = file_name+".pkl"
    with open(model_save_path, 'wb') as file:
        pickle.dump(model, file)

In [30]:
file_name  = 'saved_model_for_App'
save_model(text_clf_lsvc, file_name)

In [31]:
def load_model(file_name):
    model_load_path = file_name+".pkl"
    with open(model_load_path, 'rb') as file:
        unpickled_model = pickle.load(file)
    return unpickled_model

In [38]:
loaded_model = load_model(file_name)
y_pred = loaded_model.predict([train_df['detokenized'][2]])

In [39]:
y_pred

array([2], dtype=int64)

In [37]:
train_df['detokenized'][2]

'researcher say three year act climate change late'

### Training on clean message without retweet

In [ ]:
# removing any repeated messages while keeping the first of those messages
new_df = train_df.drop_duplicates(subset=['detokenized'], keep='first')

In [ ]:
# defing the x and y varible
X= new_df['detokenized']
y=new_df['sentiment']
# slpitting the X and y into train and test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Feeding the training data through the Linear SVC pipeline
text_clf_lsvc.fit(X_train, y_train)
# Form a prediction set
dtknz_pred = text_clf_lsvc.predict(X_test)
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,dtknz_pred))
# Print a classification report
print(metrics.classification_report(y_test,dtknz_pred))

In all of the above Trainings, the model is better at predicting class 1, this is due to imbalance in classes.

## Downsampling the majority classes

Since the class Pro(1),News (2) and Neutral(0) has so many observations, thier sizes will be reduced by taking a small random subset of observations to match the size of the class Anti(-1). 

In [ ]:
from sklearn.utils import resample

In [ ]:
#assigning the sentiments
Pro = train_df[train_df['sentiment']==1]
Anti = train_df[train_df['sentiment']== -1]
Neutral = train_df[train_df['sentiment']==0]
News = train_df[train_df['sentiment']== 2]

In [ ]:
# Downsample majority
Pro_downsampled = resample(Pro,
                          replace=False, # sample without replacement (no need to duplicate observations)
                          n_samples=len(Anti), # match number in minority class
                          random_state=27) # reproducible results
Neutral_downsampled = resample(Neutral,
                          replace=False, # sample without replacement (no need to duplicate observations)
                          n_samples=len(Anti), # match number in minority class
                          random_state=27) # reproducible results
News_downsampled = resample(News,
                          replace=False, # sample without replacement (no need to duplicate observations)
                          n_samples=len(Anti), # match number in minority class
                          random_state=27) # reproducible results



# Combine downsampled majority class with minority class
downsampled = pd.concat([Pro_downsampled,Neutral_downsampled,News_downsampled,Anti])

In [ ]:
temp = downsampled.groupby('sentiment').count()['message'].reset_index().sort_values(by='message',ascending=False)
temp['percentage'] = (temp['message']/temp['message'].sum())*100
temp

Now the sentiments are balanced

Training on downsampled clean message

In [ ]:
downsampled.head(1)

In [ ]:
# defing the x and y varible
X= downsampled['detokenized']
y=downsampled['sentiment']
# slpitting the X and y into train and test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Linear SVC:
text_clf_lsvc = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])
text_clf_lsvc.fit(X_train, y_train)
# Form a prediction set
predictions = text_clf_lsvc.predict(X_test)
print(metrics.confusion_matrix(y_test,predictions))
# Print a classification report
print(metrics.classification_report(y_test,predictions))

The score has improved a lot, now all sentiments have better score above 74%

In [ ]:
X_test = test_df['detokenized']
down_s_pred= text_clf_lsvc.predict(X_test)

In [ ]:
down_s_pred = pd.DataFrame(data=down_s_pred,
                                 columns=['sentiment'],
                                 index=test_df['tweetid'])
down_s_pred.reset_index(inplace=True)

In [ ]:
down_s_pred.to_csv(r"C:\Users\elton\Documents\Jm_team\Classification_Predict_Package\predictions_SVC\down_s_pred.csv", index=False)

### Now Downsampling the dataframe(new_df) without the retweets and train the model.

In [ ]:
#assigning the sentiments
Pro = new_df[new_df['sentiment']==1]
Anti = new_df[new_df['sentiment']== -1]
Neutral = new_df[new_df['sentiment']==0]
News = new_df[new_df['sentiment']== 2]

In [ ]:
# Downsample majority
Pro_downsampled = resample(Pro,
                          replace=False, # sample without replacement (no need to duplicate observations)
                          n_samples=len(Anti), # match number in minority class
                          random_state=27) # reproducible results
Neutral_downsampled = resample(Neutral,
                          replace=False, # sample without replacement (no need to duplicate observations)
                          n_samples=len(Anti), # match number in minority class
                          random_state=27) # reproducible results
News_downsampled = resample(News,
                          replace=False, # sample without replacement (no need to duplicate observations)
                          n_samples=len(Anti), # match number in minority class
                          random_state=27) # reproducible results



# Combine downsampled majority class with minority class
downsampled2 = pd.concat([Pro_downsampled,Neutral_downsampled,News_downsampled,Anti])

In [ ]:
temp = downsampled2.groupby('sentiment').count()['message'].reset_index().sort_values(by='message',ascending=False)
temp['percentage'] = (temp['message']/temp['message'].sum())*100
temp

it is observable that the classes have reduced since the retweets were dropped

In [ ]:
# defing the x and y varible
X= downsampled2['detokenized']
y=downsampled2['sentiment']
# slpitting the X and y into train and test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Linear SVC:
text_clf_lsvc = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])
text_clf_lsvc.fit(X_train, y_train)
# Form a prediction set
predictions = text_clf_lsvc.predict(X_test)
print(metrics.confusion_matrix(y_test,predictions))
# Print a classification report
print(metrics.classification_report(y_test,predictions))

the model now is perfoming worse, this may be because the model is underfed.

## Using SMOTE Algorithm

In [ ]:
from sklearn.feature_extraction.text import *
tfidf_vectorizer = TfidfVectorizer(min_df=100)

In [ ]:
#import SMOTE module from imblearn library 
# pip install imblearn (if you don't have imblearn in your system) 
from imblearn.over_sampling import SMOTE
# defing the x and y varible
X= train_df['detokenized']
y=train_df['sentiment']
 
sm = SMOTE(random_state = 2) 

X = tfidf_vectorizer.fit_transform(X)
X, y = sm.fit_sample(X, y.ravel()) 



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
text_clf_lsvc = Pipeline([('clf', LinearSVC())])
text_clf_lsvc.fit(X_train, y_train)
# Form a prediction set
predictions = text_clf_lsvc.predict(X_test)
print(metrics.confusion_matrix(y_test,predictions))
# Print a classification report
print(metrics.classification_report(y_test,predictions))

In [ ]:
#predicting the Time_from_Pickup_to_Arrival with unseen x test
X_test = test_df['detokenized']
X_test = tfidf_vectorizer.fit_transform(X_test)
#smote_pred = text_clf_lsvc.predict(X_test)

In [ ]:
print(X_test)

Smote without retweets

In [ ]:
#import SMOTE module from imblearn library 
# pip install imblearn (if you don't have imblearn in your system) 
from imblearn.over_sampling import SMOTE
# defing the x and y varible
X= new_df['detokenized']
y=new_df['sentiment']
 
sm = SMOTE(random_state = 2) 

X = tfidf_vectorizer.fit_transform(X)
X, y = sm.fit_sample(X, y.ravel()) 




In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
text_clf_lsvc = Pipeline([('clf', LinearSVC())])
text_clf_lsvc.fit(X_train, y_train)
# Form a prediction set
predictions = text_clf_lsvc.predict(X_test)
print(metrics.confusion_matrix(y_test,predictions))
# Print a classification report
print(metrics.classification_report(y_test,predictions))